In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation

In [ ]:
gd = [0.06, 0.223, 0.372, 0.196, -0.135, 0.426, 8.37]
gd_1 = [0.0508,1.1461,-0.3277,-0.2838,-0.4065,0.0431,8.48847328196] #X_1
gd_2 = [0.4539,-0.9662,-0.1322,-0.0,-0.7994,0.0441,8.12947125898] #X_2
gd_3 = [0.1704,-0.0947,-0.4962,-0.0,0.1037,0.0109,7.89770339493] #X_3
gd_4 = [0.8576,-0.8836,2.9865,0.0,-1.8508,-0.0143,8.33126119382] #X_4
gd_5 = [1.1957,-0.693,0.0005,-0.0,-0.6442,0.0293,8.21512214126] #X_5
ed = [-2.82, -1.104, 0.827, 0.96, 9.471]
ed_1 = [-2.147,-0.398,0.0,1.3791,9.6361887613] #X_1
ed_2 = [-0.4101,-0.9549,0.0,0.7409,8.82618121612] #X_2
ed_3 = [-1.7345,1.102,0.0,0.2559,7.82148744848] #X_3
ed_4 = [-3.9587,0.1358,0.8394,1.6839,9.56795015331] #X_4
ed_5 = [-1.8173,1.0672,0.0,-0.4283,9.10474754996] #X_5
gp = [-3.094, -7.545, 11.025, 9.011]
gp_1 = [-5.4733,-5.8343,10.7325,9.6022] #X_1
gp_2 = [-4.6902,-2.5154,16.9676,0.1287] #X_2
gp_3 = [-2.2335,-8.7922,6.2464,13.6984] #X_3
gp_4 = [-3.0828,-5.5514,10.5674,8.1592] #X_4
gp_5 = [-5.0835,-16.6975,7.5219,23.5545] #X_5

In [ ]:
FEATURES = [
    "gd",
    "ed",
    "gp",
#    "isr",
#    "views", 
    "large", 
     "top", 
#    "speed_level",
    "speed_level 1",
    "speed_level 2",
    "speed_level 3",
    "price",
#    "price_level 1",
#    "price_level 2",
#    "price_level 3",
]

In [ ]:
def get_speed_level(speed_level):
    if speed_level == 1:
        #return  [1]
        return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        #return [0.4]
        return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        #return [0.1]
        return [0,0,1]
        #print X[-1], speed_level
    else:
        #return [0]
        return [0,0,0]
        #print X[-1], speed_level

In [ ]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        X+=[[]]
        #gd 0-7
        X[-1]+=[doc['degree'].count(u'Профессор')]
        X[-1]+=[int(doc['top'])]
        X[-1]+=[len(doc['illnesses'])]
        X[-1]+=[len(doc['languages'])]
        X[-1]+=[int(doc['h_index'])]
        X[-1]+=[int(doc['experience'])]
        X[-1]+=[1]
        #ed 7-12
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        X[-1]+=[len(list(set(ids_diad)))]
        X[-1]+=[len(doc['procedures']['diagnostics'])]
        X[-1]+=[len(list(set(ids_oper)))]
        X[-1]+=[len(doc['procedures']['operations'])]
        X[-1]+=[1]
        #gp 12-16
        X[-1]+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
        X[-1]+=[len(all_proc)] #gen_proc_cli
        X[-1]+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
        X[-1]+=[len(clinic['procedures']['operations'])] #gen_oper_cli
        #
        c_id = int(clinic['country_id'])
#        if c_id == 1:
#            X[-1]+=[1]
#        else:
#            X[-1]+=[0]
        #X[-1]+=[int(clinic['views'])]
        X[-1]+=[int(clinic['large'])]
        X[-1]+=[int(clinic['top'])]
        #X[-1]+=[int(clinic['speed_level'])]
        X[-1]+=get_speed_level(int(clinic['speed_level']))
        #X[-1]+=[round(1-int(clinic_estim['price'])/10.0,4)]
        X[-1]+=get_speed_level(int(clinic['price_level']))
        #X[-1]+=[round(int(clinic['price_level'])/3.0,4)]
    if clinic_estim != []:
        y=[int(clinic_estim['rating'])] * len(all_doc)
        return [X, y]
    else:
        return X

In [ ]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    #t_cleaned_data = [extract_data(i,[]) for i in t_data]
    X = sum([i[0] for i in t_cleaned_data],[])
    y = sum([i[1] for i in t_cleaned_data],[])
    return X,y

In [ ]:
def get_X_from(t_data):
    t_cleaned_data = []
    clinic_names = []
    ids = []
    for i in t_data:
        if i['doctors']!=[]:
            t_cleaned_data += [extract_data(i,[])]
            clinic_names += [unicode(i['name_ru'])] * len(i['doctors'])
            ids += [int(i['id'])] * len(i['doctors'])
    X = sum([i for i in t_cleaned_data],[])
    return X, clinic_names, ids

In [ ]:
def normale(mat):
    for i in range(0,len(mat[0])-1):
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [ ]:
def pack(X, gd, ed, gp):
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:7])),5)],
            [round(sum(np.array(ed)*np.array(x[7:12])),5)],
            [round(sum(np.array(gp)*np.array(x[12:16])),5)],
            x[16:]
        ],[])]
    return new_X

In [ ]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=True, normalize=True)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X_all, y_all = [], []

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[2]['clinics']
with open('./max/меланома_все.json') as data_file: #2
    cancer_data = json.load(data_file)[0]['clinics']
X_1, y_1 = get_X_y_from(cancer_data)
X_all+=X_1
y_all+=y_1
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[0]['clinics']
with open('./max/рак_груди_все.json') as data_file: #0
    cancer_data = json.load(data_file)[0]['clinics']
X_2, y_2 = get_X_y_from(cancer_data)
X_all+=X_2
y_all+=y_2
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[4]['clinics']    
with open('./max/рак_простаты_все.json') as data_file: #4
    cancer_data = json.load(data_file)[0]['clinics']
clinic_ids = [i['id'] for i in estims_data]
estims_data.pop(clinic_ids.index('0'))
X_3, y_3 = get_X_y_from(cancer_data)
X_all+=X_3
y_all+=y_3

with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']     
with open('./max/рак_шейки_матки_все.json') as data_file: #1
    cancer_data = json.load(data_file)[0]['clinics']
X_4, y_4 = get_X_y_from(cancer_data)
X_all+=X_4
y_all+=y_4
    
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[3]['clinics']         
with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_5, y_5 = get_X_y_from(cancer_data)
X_all+=X_5
y_all+=y_5

normale(X_all)
X_all=pack(X_all,gd, ed, gp)
normale(X_all)

normale(X_1)
X_1=pack(X_1,gd_1, ed_1, gp_1)
normale(X_1)

normale(X_2)
X_2=pack(X_2,gd_2, ed_2, gp_2)
normale(X_2)

normale(X_3)
X_3=pack(X_3,gd_3, ed_3, gp_3)
normale(X_3)

normale(X_4)
X_4=pack(X_4,gd_4, ed_4, gp_4)
normale(X_4)

normale(X_5)
X_5=pack(X_5,gd_5, ed_5, gp_5)
normale(X_5)

In [ ]:
for i in range(1,7):
    print i
    regr = process_with(X_4, y_4, info=True, ts=i/10.0)
    print

In [ ]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0] #, i[1]
print regr.intercept_

In [ ]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
#with open('./max/меланома_все.json') as data_file: #2
#with open('./max/рак_груди_все.json') as data_file: #0
with open('./max/рак_простаты_все.json') as data_file: #4
#with open('./max/рак_шейки_матки_все.json') as data_file: #1
#with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_test, names, ids = get_X_from(cancer_data)
normale(X_test)
X_test = pack(X_test,gd_3, ed_3, gp_3)
normale(X_test)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
results=[]
for i in range(0, len(X_test)):
    results += [(regr.predict(X_test[i])[0], names[i])]

res=[]
for n in set(names):
    indices = [j for j, x in enumerate(results) if x[1] == n]
    m = max([i[0] for i in results[min(indices):max(indices)+1]])
    res+=[(m,n, ids[indices[0]])]

for r in res:
#for r in sorted(results, reverse=True):
    #print "%2.2f \t %s" % (r[0], r[1])
    print r[0]

In [ ]:
pos = [10 - sorted(res, reverse=True).index(x) for x in res]
for p in pos:
    print p

In [ ]:
def seq_procent(input_array):
    new_pos=np.array(input_array)
    straight = np.array([i for i in range(max(new_pos), min(new_pos)-1, -1)])
    inversions = abs(straight - new_pos)
    while not np.array_equal(inversions, np.zeros(len(inversions))):
        new_pos=np.delete(new_pos, inversions.argmax())
        new_pos = [10 - sorted(new_pos, reverse=True).index(x) for x in new_pos]
        straight = np.array([i for i in range(max(new_pos), min(new_pos)-1, -1)])
    return 100*len(new_pos)/float(len(input_array))

In [ ]:
b1 = [10,7,9,8]
b2= [10, 9, 8]
len(set(b1).intersection(b2[0:2]))

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_2)):    
    print "Real: %f \t Predicted: %f" %(y_2[i], regr.predict(X_2[i]))
    print X_2[i]
    print
    #print y_all[i]
    #print regr.predict(X_all[i])[0]

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    k=0
    for X in [X_1, X_2, X_3, X_4, X_5]:
        k += 1
        plt.plot(sorted([i[f] for i in X]))
        #plt.show()
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.clf()
    f+=1

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(sorted(mass[l:r]), label="X%d"%k)
        plt.ylim([min(mass),1])
        legend = plt.legend(loc=4, shadow=True)
        #plt.show()
    plt.savefig('%s.png' % (feature))
    plt.clf()
    f+=1

In [ ]:
plt.plot([np.linalg.norm(i) for i in X_all], color='black')
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(y_all[l:r], [i[f] for i in X_all[l:r]], label="X%d"%k)
        plt.ylim([min(mass),1])
        plt.xlim([0,11])
        legend = plt.legend(loc=4, shadow=True)
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.show()
        plt.clf()
    f+=1

In [ ]:
k=0
r=0
l=0
for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
    l=r
    r+=length
    k+=1
    plt.plot(y_all[l:r], [np.linalg.norm(i[2]) for i in X_all[l:r]], label="X%d"%k)
    legend = plt.legend(loc=4, shadow=True)
    plt.xlim([0,11])
    plt.savefig('gp norms - X%d.png'%k)
    plt.clf()

In [ ]:
print FEATURES[2]
print [i[0] for i in X_1]
print [i for i in y_1]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)

In [ ]:
x = [i[-2] for i in X_all]
y_top = []
y_n_top = []
for l in range(0, len(x)):
    if x:
        y_top += [y[l]]
    else:
        y_n_top += [y[l]]
#for l in range(0, len(x)):
    #print x[l], y[l]

In [ ]:
for i in range(1,11):
    print i, y_top.count(i)

In [ ]:
plt.plot( x, y_all, 'ro')
plt.show()

In [ ]:
X_all[20]

In [ ]:
#for table filling
res_X = []
res_y = []
#for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
for X,y in [(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            #print res_X[-1], res_y[-1]
        except Exception, err:
            #print err, i, indices
            pass
for x in res_X:
    print x

In [ ]:
pl = [ round(3*i[-1],0) for i in X_all]
plt.hist(pl, bins=4)
plt.title("Price level distribution")
plt.xlabel("Price level")
plt.ylabel("Frequency")

plt.show()

In [ ]:
list(set(pl))

In [ ]:
set([i[-1] for i in X_all])

In [ ]:
y_all[13+17:13+17+12]

In [ ]:
indices = [i for i, x in enumerate(y_1) if x == 2]
print indices
for ind in indices:
    print regr.predict(X_1[ind])

print max(regr.predict(X_1[min(indices):max(indices)+1]))

In [ ]:
l=0
r=0
all_pos = []
for i in [7, 5, 7, 10, 6]:
    l=r
    r+=i
    all_pos += [10-sorted(res_X[l:r], reverse=True).index(x) for x in res_X[l:r]]

In [ ]:
import xlsxwriter

workbook = xlsxwriter.Workbook('Positions.xlsx')
#list 1
worksheet = workbook.add_worksheet("Рак кожи")
worksheet.freeze_panes(1, 0)
col = 0
row = 0

temp = [u"Клиника", u"Оценка (все)", u"Позиция (все)", u"Оценка (свои)", u"Позиция (свои)", u"Оценка (свои+vec pl)", u"Позиция (свои+vec pl)", u"Оценка врача", u"Позиция врача", u"Позиция на сайте"]
for i in temp:
    worksheet.write(row, col, i)
    col+=1

row=1
col=0

# num and clinic name for any model
for i, el in enumerate(zip(res,pos)):
    worksheet.write(row, 0, i)
    print el[0][1]
    worksheet.write_string(row, 1, el[0][1].encode('utf-8').replace('ё', 'е'))
    row+=1

#rate and pos of model
for el in zip(res,pos):    
    worksheet.write(row, 2, el[0][0])
    worksheet.write(row, 3, el[1])

workbook.close()

In [ ]:
pos_for_x[0:3]

In [ ]:
#creare xlsx file for model trained on ALL data

import xlsxwriter

#pre
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
#for table filling
res_X = []
res_y = []
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            #print res_X[-1], res_y[-1]
        except Exception, err:
            #print err, i, indices
            pass

l=0
r=0
all_pos = []
seq_pr = []
top_3 = []
top_3_strict = []
for i in [7, 5, 7, 10, 6]:
    l=r
    r+=i
    pos_for_x = [10-sorted(res_X[l:r], reverse=True).index(x) for x in res_X[l:r]]
    all_pos += pos_for_x
    seq_pr += [seq_procent(pos_for_x)]
    top_3 += [len(set([10,9,8]).intersection(pos_for_x[0:3]))]
    top_3_strict += [sum(pos_for_x[0:3] == np.array([10,9,8]))]
    
    
workbook = xlsxwriter.Workbook('Model_speed_level.xlsx')

#list 1
worksheet = workbook.add_worksheet("coef")
worksheet.write(0, 0, "f")
worksheet.write(0, 2, "rating")

row = 1
col = 2

for num, feat in f:
    worksheet.write(row, col, feat)
    worksheet.write(row, col+1, num)
    row+=1

worksheet.write(row, col, "intercept")
worksheet.write(row, col+1, regr.intercept_)
row+=2
col=0

r=row

worksheet.write(row, col, "GD")
worksheet.write(row, col+2, "specialist")
row+=1

f_ = zip(map(lambda x: round(x, 4), gd), ["deg", "top", "illn", "lang", "h_index", "exp", "intercept"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1

row = r
col += 4
worksheet.write(row, col, "ED")
worksheet.write(row, col+2, "experience")
row+=1

f_ = zip(map(lambda x: round(x, 4), ed), ["top_diag", "gen_diag", "top_oper", "gen_oper", "intercept"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1
    

row = r
col += 4
worksheet.write(row, col, "GP")
worksheet.write(row, col+2, "experience")
row+=1

f_ = zip(map(lambda x: round(x, 4), gp), ["top_diag", "gen_diag", "top_oper", "gen_oper"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1

    
#list 2
worksheet = workbook.add_worksheet("tabl")
worksheet.freeze_panes(1, 0)
col = 0
row = 0

for i in [u"Болезнь", u"Клиника", u"Оценка врача", u"Оценка модели", u"Позиция врача", u"Позиция модели", u"Разница оценок врача и модели", u"Разница позиций врача и модели"]:
    worksheet.write(row, col, i)
    col+=1

col=0
row=1
temp =[u"Рак кожи"] * 7 + [u"Рак груди"] * 5 + [u"Рак простаты"] * 7 + [u"Рак шейки матки"] * 10 + [u"Рак щитовидки"] * 6
for i in temp:
    worksheet.write(row, col, i)
    row+=1

col=1
row=1
temp = [u"Ихилов", u"Текнон", u"Анадолу", u"Ассута", u"Клиника Женераль-Болье", u"Мемориал", u"Фортис", u"Ассута", u"Анадолу", u"Ихилов", u"Медиполь", u"Фортис", u"Дортмунд", u"Текнон", u"Ихилов", u"Ассута", u"Анадолу", u"Самсунг", u"Мемориал", u"Ихилов", u"Золинген", u"Анадолу", u"Текнон", u"Самсунг", u"Медиполь", u"Фортис", u"Нордвест", u"Ассута", u"Мемориал", u"Ихилов", u"Текнон", u"Анадолу", u"Ассута", u"Мемориал", u"Фортис"]
for i in temp:
    worksheet.write(row, col, i)
    row+=1
    
col=2
row=1
temp = [10, 9, 8, 6, 4, 3, 1, 9, 7, 5, 4, 2, 9, 8, 6, 5, 4, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 6, 2, 1]
for i in temp:
    worksheet.write(row, col, i)
    row+=1

col=3
row=1
for i in res_X:
    worksheet.write(row, col, i)
    row+=1
    
col=4
row=1
temp = [10, 9, 8, 7, 6, 5, 4, 10, 9, 8, 7, 6, 10, 9, 8, 7, 6, 5, 4, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5]
for i in temp:
    worksheet.write(row, col, i)
    row+=1

col=5
row=1
for i in all_pos:
    worksheet.write(row, col, i)
    row+=1


col=6
row=1
for i in range(0, len(temp)):
    worksheet.write_formula(row, col, '=C%d-D%d'%(row+1,row+1))
    worksheet.write_formula(row, col+1, '=E%d-F%d'%(row+1,row+1))
    worksheet.write_formula(row, col+2, '=ABS(G%d)'%(row+1))
    worksheet.write_formula(row, col+3, '=ABS(H%d)'%(row+1))
    row+=1

for l,r in [(2,8),(9,13),(14,20),(21,30),(31,36)]:
    worksheet.write_formula('K%d' % r, '=AVERAGE(I%d:I%d)'%(l,r))
    worksheet.write_formula('L%d' % r, '=AVERAGE(J%d:J%d)'%(l,r))

#list 3
worksheet = workbook.add_worksheet("stat")
worksheet.freeze_panes(1, 0)
col = 1
row = 0

for i in [ u"Среднее по оценке", u"Среднее по позиции", u"% последовательности", u"топ-3 с 10", u"топ-3"]:
    worksheet.write(row, col, i)
    col+=1

col=0
row=1
for i in [u"Рак кожи", u"Рак груди", u"Рак простаты", u"Рак шейки матки", u"Рак щитовидки", "", u"Итого"]:
    worksheet.write(row, col, i)
    row+=1

col=1    
row=1
for i in [8, 13,20, 30, 36]:
    worksheet.write(row, col, "='tabl'!K%d"%(i))
    worksheet.write(row, col+1, "='tabl'!L%d"%(i))
    row+=1

col=3
row=1
for i in zip(seq_pr,top_3_strict, top_3):
    worksheet.write(row, col, i[0])
    worksheet.write(row, col+1, i[1])
    worksheet.write(row, col+2, i[2])
    row+=1
    
row+=1
col=1
for i in ['B', 'C', 'D', 'E', 'F']:
    worksheet.write_formula(row, col, '=AVERAGE(%s2:%s6)'%(i,i))
    col+=1

workbook.close()

In [ ]:
#for table filling

res_X = []
res_y = []
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            print res_X[-1], res_y[-1]
        except Exception, err:
            #print err, i, indices
            pass
#for x in res_X:
#    print x